# Class preparation Data

## INSEE

Les données sources de l'INSEE proviennent de [Data Gouv](https://www.data.gouv.fr/fr/datasets/base-sirene-des-entreprises-et-de-leurs-etablissements-siren-siret/)

- communes_insee:
    - Le fichier source pour les communes se trouvent à cette [URL](https://www.insee.fr/fr/information/3720946)
    - Le notebook pour reconstituer le csv est disponible a cette [URL](https://github.com/thomaspernet/InseeInpi_matching/blob/dev_thomas/Notebooks_matching/programme_matching/Source_intermediates.ipynb). ⚠️ Repo privé + branche
- voie:
    - Le fichier source pour les communes se trouvent à cette [URL](https://www.sirene.fr/sirene/public/variable/libelleVoieEtablissement)
    - Le notebook pour reconstituer le csv est disponible a cette [URL](https://github.com/thomaspernet/InseeInpi_matching/blob/dev_thomas/Notebooks_matching/programme_matching/Source_intermediates.ipynb). ⚠️ Repo privé + branche
- upper_word:
    - La liste des upper word (stop word capitalisé) provient de la librarie [NLTK](https://www.nltk.org/) avec un ajout manuel.
    
## INPI 

Les données de l'INPI proviennent de ses différents Notebooks:

- inpi_etb

## Normalisation du fichier INPI.

Le fichier INPI doit contenir un seul fichier gz avant d'être ingéré par le programme. Le fichier va être importé dans un format Dask, ce qui permet de paralléliser les calcules et bien sur d'éviter les problèmes de mémoire. 

La normalisation du fichier de l'INPI se fait en plusieurs étapes:

- 1) Exclusion des observations contenant des NaN pour chacune des variables candidates, à savoir:
    - Adresse_Ligne1
    - Adresse_Ligne2
    - Adresse_Ligne3
    - Code_Postal
    - Ville
    - Code_Commune
- 2) Extraction des SIREN a SIRETISER -> cela evite d'utiliser toute la base INSEE pour la sirétisation. I.e Speedup le process
- 3) Calcule du nombre de SIRET par SIREN via la fonction `nombre_siret_siren`
- 4) Normalisation de la variable commune via la fonction `clean_commune`
    - Extraction des digits dans la ville. En effet, certaines communes incluent l'arrondissement dans la variable.
    - Extraction des caractères spéciaux et espaces
    - Capitalisation du nom de la commune
    - Matching avec le fichier commune pour avoir le nom de la commune de l'INSEE.
- 5) Préparation de l'adresse via la fonction `prepare_adress`
    - Concatenation des variables `Adresse_Ligne1` + `Adresse_Ligne2` + `Adresse_Ligne3`
    - Normalisation de la variable concatenée -> Extraction des caractères speciaux, espace, digit puis capitalisation
    - Extraction de tous les stop words du fichier `upper_word`
    - Split de chaque mot restant de l'adresse 
    - Creation du regex de la forme suivante:  `MOT1$|MOT2$` 
    - Extration des digits:
        - Première variable avec le premier digit
        - Seconde variable avec une liste de digit et jointure -> DIGIT1|DIGIT2
    - Merge avec le fichier `voie` pour obtenir le type de voie de l'INSEE
    - Calcule du nombre de digit dans l'adresse
        - Si len inférieure a 2, alors NaN. C'est une variable utlisée pendant le matching des règles spéciales
    - Creation d'une variable `index` correspondant à l'index du dataframe. Indispensable
 
Le fichier est sauvegardé en format gz, et dans un table SQL
    - inpi_etb_stock_0.gz
    - inpi_origine.db
    
Un appercu de la table est disponible via cette application `App_inpi`.

## Normalisation du fichier INSEE

Pour l'étape de siretisation, les variables candidates sont les suivantes:

- 'siren',
- 'siret',
- "etablissementSiege",
- "etatAdministratifEtablissement",
- "numeroVoieEtablissement",
- "indiceRepetitionEtablissement",
- "typeVoieEtablissement",
- "libelleVoieEtablissement",
- "complementAdresseEtablissement",
- "codeCommuneEtablissement",
- "libelleCommuneEtablissement",
- "codePostalEtablissement",
- "codeCedexEtablissement",
- "libelleCedexEtablissement",
- "distributionSpecialeEtablissement",
- "libelleCommuneEtrangerEtablissement",
- "codePaysEtrangerEtablissement",
- "libellePaysEtrangerEtablissement",
- "dateCreationEtablissement"

Comme pour le fichier de l'INPI, le fichier csv est importé en Dask Dataframe. Les étapes sont les suivantes:

- 1) Filtre les SIREN à sirétiser uniquement
- 2) Filtre la date limite à l'INSEE. Cette étape sert essentiellement pour siretiser les bases de stocks. Cela évite d'utiliser des valeurs "dans le future" -> inconnu à l'INPI
- 3) Remplacement des "-" par des " " dans la variable `libelleCommuneEtablissement`
- 4) Extraction des digits en format liste de la variable `libelleVoieEtablissement`
- 5) Calcule du nombre de SIRET par SIREN
- 6) Calcule du nombre de digit dans la variable `libelleCommuneEtablissement`
    - Si len inférieure a 2, alors NaN. C'est une variable utlisée pendant le matching des règles spéciales
    
Le fichier est sauvegardé en format gz, et dans un table SQL
    - insee_2017_SIZE.gz
    - App_insee.db
    
Un appercu de la table est disponible via cette application `App_insee`.

In [ ]:
from inpi_insee import preparation_data

%load_ext autoreload
%autoreload 2

In [ ]:
param = {
    'communes_insee': r'data\input\Parameters\communes_france.csv',
    'upper_word':r'data\input\Parameters\upper_stop.csv',
     "voie":r'data\input\Parameters\voie.csv',
    'insee': r"data\RawData\INSEE\Stock\ETS\StockEtablissement_utf8.csv",
    'inpi_etb': r"data\RawData\INPI\Stock\initial_ETS.gz",
    'date_end':"2018-01-01"
}
prep_data = preparation_data.preparation(param)

In [ ]:
%%time
prep_data.normalize_inpi(save_gz = True, save_sql = True)

In [ ]:
%%time
prep_data.normalize_insee(siren_inpi_gz = 
                          r'data\input\SIREN_INPI\inpi_etb_stock_4824158.gz',
                                save_gz = True, save_sql = True)

# Test

In [ ]:
test.head()

In [ ]:
test.groupby('len_digit_address_insee')['len_digit_address_insee'].count()

In [ ]:
test.loc[lambda x: 
         x['siren'].isin(['752085324'])
         & (len(x['special_digit'])>=2)
        ]

In [ ]:
test.assign(
        special_digit = lambda x:
        x['libelleVoieEtablissement'].str.findall(r"(\d+)").apply(
        lambda x:'&'.join([i for i in x]))
        )